In [14]:
import sys
sys.path.append("./")
from quaos.circuits.utils import solve_modular_linear
from quaos.circuits import Gate
from quaos.hamiltonian import symplectic_pauli_reduction, pauli_reduce, random_pauli_hamiltonian
from quaos.paulis import PauliSum

In [16]:
def find_allowed_target(pauli_sum, target_pauli_list, ):
    pauli_list = [target_pauli_list[i][0] for i in range(len(target_pauli_list))]
    pauli_list = str_to_int(pauli_list)
    string_indices = [target_pauli_list[i][1] for i in range(len(target_pauli_list))]
    qudit_indices = [target_pauli_list[i][2] for i in range(len(target_pauli_list))]

    dims = pauli_sum.dimensions 
    combined_indices = list(zip(string_indices, qudit_indices))
    index_dict = defaultdict(list)

    for idx, tup in enumerate(combined_indices):
        index_dict[tup].append(idx)

    underdetermined_pauli_indices = [combined_indices[idxs[0]] for idxs in index_dict.values() if len(idxs) > 1]
    underdetermined_pauli_options = []
    for indices in underdetermined_pauli_indices:
        options = []
        for i in range(len(target_pauli_list)):
            if target_pauli_list[i][1] == indices[0] and target_pauli_list[i][2] == indices[1]:
                if target_pauli_list[i][0] not in options:
                    options.append(target_pauli_list[i][0])
        underdetermined_pauli_options.append(options)
    underdetermined_pauli_options = [str_to_int(l) for l in underdetermined_pauli_options]

    determined_pauli_indices = [combined_indices[idxs[0]] for idxs in index_dict.values() if len(idxs) == 1]

    options_matrix = np.empty([pauli_sum.n_paulis(), pauli_sum.n_qudits()], dtype=object)
    for i in range(pauli_sum.n_paulis()):
        for j in range(pauli_sum.n_qudits()):
            if (i, j) in underdetermined_pauli_indices:
                options_matrix[i, j] = underdetermined_pauli_options[underdetermined_pauli_indices.index((i, j))]
            elif (i, j) in determined_pauli_indices:
                options_matrix[i, j] = [pauli_list[determined_pauli_indices.index((i, j))]]
            else:
                options_matrix[i, j] = [0, 1, 2, 3]
    
    flag_matrix = np.zeros([pauli_sum.n_paulis(), pauli_sum.n_qudits()], dtype=int)  # flag matrix to track which indices are determined 
    # 0 for not determined, 1 for underdetermined, -1 for determined
    for idx, tup in enumerate(combined_indices):
        if tup in underdetermined_pauli_indices:
            flag_matrix[tup[0], tup[1]] = -1  # -1 for underdetermined
        else:
            flag_matrix[tup[0], tup[1]] = 1  # 1 for determined

    spm = pauli_sum.symplectic_product_matrix()
    
    possible_targets = []
    for combo in product(*options_matrix.flatten()):
        combo = np.reshape(combo, (pauli_sum.n_paulis(), pauli_sum.n_qudits()))
        pauli_sum_candidate = matrix_to_pauli_sum(combo, pauli_sum.weights, pauli_sum.phases, dims)
        candidate_spm = pauli_sum_candidate.symplectic_product_matrix()

        if np.all(spm == candidate_spm):
            possible_targets.append(pauli_sum_candidate)
    return possible_targets



In [17]:
def standard_form_to_basis(pauli_sum: PauliSum) -> tuple[PauliSum, list[tuple[int, int, int]]]:
    new_ps = pauli_sum.copy()
    multiplied_paulis = []

    for q in range(pauli_sum.n_qudits()):
    
        ixs, izs = find_ix_iz(new_ps, q)
        if len(ixs) == 0 and len(izs) == 0:
            print('No ix or iz found for qudit ', q)
            continue
        ixs = ixs[0] if len(ixs) > 0 else None
        izs = izs[0] if len(izs) > 0 else None
        if ixs is None:
            print('No ix found for qudit ', q)
        if izs is None:
            print('No iz found for qudit ', q)
        for p in range(new_ps.n_paulis()):  # min( , 2 * new_ps.n_qudits())
            if new_ps[p].x_exp[q] != 0 and ixs is not None and ixs != p:
                n = solve_modular_linear(new_ps[p].x_exp[q], new_ps[ixs].x_exp[q], new_ps.dimensions[q])
                new_ps[p] = new_ps[p] * new_ps[ixs]**n
                multiplied_paulis.append((p, ixs, n))
            if new_ps[p].z_exp[q] != 0 and izs is not None and izs != p:
                n = solve_modular_linear(new_ps[p].z_exp[q], new_ps[izs].z_exp[q], new_ps.dimensions[q])
                new_ps[p] = new_ps[p] * new_ps[izs]**n
                multiplied_paulis.append((p, izs, n))
    return new_ps, multiplied_paulis


def multiply_paulis(pauli_sum: PauliSum, multiplier_list: list[tuple[int, int, int]]) -> PauliSum:
    """
    Multiply the pauli strings in the pauli sum by the given multipliers.

    :param pauli_sum: The PauliSum to multiply.
    :param multiplier_list: A list of tuples (pauli_index, multiplier_index, multiplier_value) where
    the pauli at pauli_index is multiplied by the pauli at multiplier_index raised to the power of multiplier_value.

    :return: The PauliSum after multiplication.
    """
    new_pauli_sum = pauli_sum.copy()
    for p, m, n in multiplier_list:
        new_pauli_sum[p] = new_pauli_sum[p] * new_pauli_sum[m]**n
    return new_pauli_sum


def is_basis(pauli_sum: PauliSum) -> tuple[bool, list[int]]:
    ixs, izs = find_ix_iz(pauli_sum)
    if len(ixs) + len(izs) == 2 * pauli_sum.n_qudits():
        return True, ixs + izs
    elif len(ixs) + len(izs) > 2 * pauli_sum.n_qudits():
        # over complete - pick only the first 2 * n_qudits independent pauli strings
        ix_qudits = []
        ixs_ = []
        iz_qudits = []
        izs_ = []
        for ix in ixs:
            ixq = np.where(pauli_sum[ix].x_exp != 0)[0][0]
            if ixq not in ix_qudits:
                ix_qudits.append(ixq)
                ixs_.append(ix)
        for iz in izs:
            izq = np.where(pauli_sum[iz].z_exp != 0)[0][0]
            if izq not in iz_qudits:
                iz_qudits.append(izq)
                izs_.append(iz)
        return True, ixs_ + izs_
    else:
        # Check if the remaining pauli strings can be made up of the ixs and izs - if so it is an incomplete basis
        remaining = [i for i in range(pauli_sum.n_paulis()) if i not in ixs and i not in izs]
        missing_ix = []
        missing_iz = []
        for q in range(pauli_sum.n_qudits()):
            ixs_q, izs_q = find_ix_iz(pauli_sum, q)
            if len(ixs_q) == 0:
                missing_ix.append(q)
            if len(izs_q) == 0:
                missing_iz.append(q)
        for r in remaining:
            r_x_exp = pauli_sum[r].x_exp
            r_z_exp = pauli_sum[r].z_exp
            for q in missing_ix:
                if r_x_exp[q] != 0:
                    # If the pauli string has an x term on a qudit where we are missing an ix, it cannot be a basis
                    return False, []
            for q in missing_iz:
                if r_z_exp[q] != 0:
                    # If the pauli string has a z term on a qudit where we are missing an iz, it cannot be a basis
                    return False, []
        return True, ixs + izs


In [18]:
def get_gate_from_target(input_pauli_sum: PauliSum, target_pauli_sum: PauliSum) -> Gate:
    """
    Given an input PauliSum and a target PauliSum, find the gate that maps the input to the target.

    We assumer that the input_pauli_sum is in standard form, that is, it is one of the conditioned PauliSums output 
    by the symplectic reduction method.
    """

    if np.any(input_pauli_sum.symplectic_product_matrix() != target_pauli_sum.symplectic_product_matrix()):
        raise ValueError("Input and target PauliSums must have the same symplectic product matrix.")
    
    # Find linear operations that map the input PauliSum to a basis representation
    
    input_pauli_basis, multipliers = standard_form_to_basis(input_pauli_sum)
    target_pauli_basis = multiply_paulis(target_pauli_sum, multipliers)

    # We then use this basis representation to construct the gate mappings to C^dag target C plus same linear operations 
    basis_check, basis_indices = is_basis(input_pauli_basis)
    if not basis_check:
        raise ValueError("Output PauliSum is not a basis representation.")
    gate_mappings = []
    for i in range(len(basis_indices)):
        gate_mappings.append((basis_indices[i], target_pauli_basis[i]))

    return Gate("CustomGate", list(range(input_pauli_sum.n_qudits())), gate_mappings, input_pauli_sum.dimensions)

In [33]:

# random hamiltonian example
n_qudits = 6
n_paulis = 6
dimension = 2

ham = random_pauli_hamiltonian(n_paulis, [dimension] * n_qudits, mode='uniform')
circuit = symplectic_pauli_reduction(ham)
h_reduced, conditioned_hams, reducing_circuit, eigenvalues = pauli_reduce(ham)
# we  look for near symmetries in the conditioned subsectors


In [ ]:
if len(conditioned_hams) > 0:
    initial_pauli_sum = conditioned_hams[0]
else:
    initial_pauli_sum = h_reduced

(-2+2.4492935982947064e-16j)|x0z0 x0z0 | 0 
(-1+1.2246467991473532e-16j)|x0z1 x0z0 | 0 
(1+0j)                      |x1z0 x0z0 | 0 
(1+0j)                      |x1z0 x0z1 | 0 
(1+0j)                      |x0z0 x1z0 | 0 



Here we specify the target Hamiltonian. We wish to identify $V$ in $H = H_0 + V$ such that $\exists$ a Clifford circuit $C$ where $II \cdots IZ$ is a symmetry of $CH_0C^\dagger$

$V = \sum_i c_i Q_i$ with $Q_i = ***\cdots **X$ are all Paulis in $CHC^\dagger$ with a leading $X$.

We want to find $C$.